In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **CARREGAMENTO DOS DADOS - Dados CNPJ EMPRESA**

In [3]:
cnpj_final = pd.read_csv('/content/drive/MyDrive/TCC - PUC/cnpj_final.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)


# **DATASET SIAFI**

Neste dataset iremos buscar as informações do "Código Município SIAFI", código utilizado pela Receita Federal do Brasil para identificar o Município na tabela de Dados Públicos de CNPJ. Como os demais dataset´s utilizam o "Código IBGE" para referenciar o município, utilizaremos este dataset SIAFI para fazer a relação das duas colunas: "Código Município SIAFI" x "Código IBGE"

------------


https://www.tesourotransparente.gov.br/ckan/dataset/lista-de-municipios-do-siafi




In [4]:
municipios_siafi = pd.read_csv('//content/drive/MyDrive/TCC - PUC/TABELA DE MUNICÍPIOS DO SIAFI.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0,decimal=',')

In [5]:
municipios_siafi.columns = ['codigo_municipio_siafi', 'cnpj_base', 'descricao_municipio', 'uf','codigo_ibge']

In [6]:
municipios_siafi.shape

(5589, 5)

In [7]:
municipios_siafi.head()

,codigo_municipio_siafi,cnpj_base,descricao_municipio,uf,codigo_ibge
0,1,"5,89363E+12",GUAJARA-MIRIM,RO,1100106
1,2,"8,4745E+13",ALTO ALEGRE DOS PARECIS,RO,1100379
2,3,"5,90313E+12",PORTO VELHO,RO,1100205
3,8,"8,47369E+13",CUJUBIM,RO,1100940
4,13,"4,09271E+12",VILHENA,RO,1100304


In [8]:
codigo_ibge_ajustado = municipios_siafi['codigo_ibge'].map(lambda x: str(x)[:-1]) #Criação da coluna codigo_ibge_ajustado para possibilitar a concatenação com a tabela covid_brasil pois a mesma não possui o digito verificador na coluna

In [9]:
municipios_siafi = pd.concat([municipios_siafi, codigo_ibge_ajustado], axis=1) # concatenação do código IBGE sem dígito verificador

In [10]:
municipios_siafi['codigo_municipio_siafi'] = municipios_siafi['codigo_municipio_siafi'].astype(int)
cnpj_final['codigo_municipio_siafi'] = cnpj_final['codigo_municipio_siafi'].astype(int)

In [11]:
municipios_siafi.columns = ['codigo_municipio_siafi', 'cnpj_base', 'descricao_municipio', 'uf','codigo_ibge', 'codigo_ibge_ajustado']

In [12]:
municipios_siafi = municipios_siafi.drop(['cnpj_base','uf'], axis=1) #eliminação das colunas desnecessárias/redundantes

In [13]:
cnpj_final = pd.merge(cnpj_final, municipios_siafi, how='left', on='codigo_municipio_siafi') # concatenação dos dataset´s pelo codigo_municipio_siafi

# **DATASET COVID**

Dados obtidos no site do Governo Federal - Ministério da Saúde - demonstrando o acumulado de casos e mortes por COVID no Brasil. O primeiro caso de COVID registrado no Brasil se deu em 26/02/2020. Já o computo de casos por município começou timidamente em 27/03/2020. Dessa forma, consideramos para o nosso estudo o dia 01/04/2020, data em que todos os municípios começaram a informar os casos registrados em suas localidades.

Para um melhor estudo do impacto da COVID no fechamento das empresas, realizamos um corte no dataset considerando a data da situação cadastral como delimitador do prazo. No total, estamos avaliando 28 meses de atividade, divididos da seguinte forma:


1.   Período 20-21 - COM COVID (14 meses) - Início: 01/04/2020 - Término: 31/05/2021
2.   Período 19-20 - SEM COVID (14 meses) - Início: 01/02/2019 - Término: 31/03/2020

Os valores constantes nos campos de COVID abaixo seguirão a seguinte orientação:
*   CNPJ´s FECHADOS (situação cadastral igual a "8"): as empresas fechadas no período de 28 meses receberão o valor correspondente de COVID na da data de fechamento da empresa. Ou seja, empresas dentro do período de COVID receberão o valor correspondente ao acumulado no dia do seu fechamento e, caso seja fora do período de COVID, o valor zero (0).
*   CNPJ´s ABERTOS (situação cadastral igual a "2"): as empresas que permenceram ABERTAS durante o período de sem COVID receberão o valor zero (0) e as empresas abertas durante o período de COVID receberão o valor acumulado até o dia 31/05/2021, data de corte do estudo.



*   Além das informações referente ao COVID, iremos incluir, à partir deste dataset, incluiremos os seguintes dados:


1.   População
2.   Estado
3.   Região

-------------

https://covid.saude.gov.br/

In [15]:
covid_brasil_1 = pd.read_csv('//content/drive/MyDrive/TCC - PUC/HIST_PAINEL_COVIDBR_2020_Parte1_31mai2021.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)
covid_brasil_2 = pd.read_csv('//content/drive/MyDrive/TCC - PUC/HIST_PAINEL_COVIDBR_2020_Parte2_31mai2021.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)
covid_brasil_3 = pd.read_csv('//content/drive/MyDrive/TCC - PUC/HIST_PAINEL_COVIDBR_Parte3_31mai2021.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)

In [16]:
covid_brasil_todas = pd.concat([covid_brasil_1, covid_brasil_2, covid_brasil_3], axis=0)

In [17]:
covid_brasil_todas.shape

(2422659, 17)

In [18]:
covid_brasil_todas.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125,2,1,0,0,NaN,NaN,NaN


In [19]:
covid_brasil_todas = covid_brasil_todas[covid_brasil_todas['codmun'] .notnull()] 
covid_brasil_todas = covid_brasil_todas[covid_brasil_todas['municipio'] .notnull()] 

In [20]:
covid_brasil_todas = covid_brasil_todas.drop(['municipio','coduf','codRegiaoSaude','nomeRegiaoSaude','semanaEpi','Recuperadosnovos','emAcompanhamentoNovos','emAcompanhamentoNovos'], axis=1) #eliminação das colunas desnecessárias/redundantes

In [21]:
covid_brasil_todas.columns = ['regiao', 'estado', 'codigo_ibge_ajustado', 'data_covid','populacaoTCU2019', 'casosAcumulado','casosNovos','obitosAcumulado','obitosNovos','interior_metropolitana']

In [22]:
data_removido = covid_brasil_todas['data_covid'].str.replace(r'\D', '')

In [23]:
codigo_ibge_data = data_removido + covid_brasil_todas['codigo_ibge_ajustado'] #coluna para permitir a identificação do município e a data de ocorrência de COVID de cada CNPJ

In [24]:
covid_brasil_todas['populacaoTCU2019'] = covid_brasil_todas['populacaoTCU2019'].astype(int)
covid_brasil_todas['casosAcumulado'] = covid_brasil_todas['casosAcumulado'].astype(int)
covid_brasil_todas['obitosAcumulado'] = covid_brasil_todas['obitosAcumulado'].astype(int)

In [25]:
casos_acumulados_porcentagem = round(((covid_brasil_todas['casosAcumulado'] * 100) / (covid_brasil_todas['populacaoTCU2019'])),3) # coluna com o índice de casos acumulados - número de casos pela população do município
obitos_acumulados_porcentagem = round(((covid_brasil_todas['obitosAcumulado'] * 100) / (covid_brasil_todas['populacaoTCU2019'])),3) # coluna com o índice de óbitos por covid acumulados - número de casos pela população do município

In [26]:
covid_brasil_todas = pd.concat([covid_brasil_todas, casos_acumulados_porcentagem, obitos_acumulados_porcentagem, codigo_ibge_data], axis=1)

In [27]:
covid_brasil_todas.columns = ['regiao', 'estado', 'codigo_ibge_ajustado','data_covid', 'populacaoTCU2019','casosAcumulado', 'casosNovos','obitosAcumulado','obitosNovos','interior_metropolitana', 'casos_acumulados_porcentagem','obitos_acumulados_porcentagem','codigo_ibge_data']

In [28]:
codigo_ibge_data_regiao = pd.DataFrame ("20210531" + cnpj_final['codigo_ibge_ajustado']) # Criação da coluna "codigo_ibge_data_regiao" para permitir a inclusão das colunas região, estado e população na tabela final

In [29]:
codigo_ibge_data_regiao.columns = ['codigo_ibge_data']

In [30]:
codigo_ibge_data_regiao  = pd.merge(codigo_ibge_data_regiao, covid_brasil_todas, how='left', on='codigo_ibge_data')

In [31]:
codigo_ibge_data_regiao = codigo_ibge_data_regiao.drop(['codigo_ibge_data','codigo_ibge_ajustado','data_covid','casosAcumulado','casosNovos','obitosAcumulado','obitosNovos','casos_acumulados_porcentagem','obitos_acumulados_porcentagem'], axis=1) #eliminação das colunas desnecessárias

In [32]:
cnpj_final = pd.concat([cnpj_final, codigo_ibge_data_regiao], axis=1)

In [33]:
cnpj_final_abertas_total = cnpj_final.query( 'codigo_situacao_cadastral=="02"')
cnpj_final_fechadas_total = cnpj_final.query( 'codigo_situacao_cadastral=="08"')

In [34]:
cnpj_final_fechadas_com_covid = cnpj_final_fechadas_total.query('data_situacao_cadastral >= "20200401" and data_situacao_cadastral <= "20210531" ') #ganhar  cnpj_final['data_situacao_cadastral'] + cnpj_final['codigo_ibge_ajustado'] 
cnpj_final_fechadas_sem_covid = cnpj_final_fechadas_total.query('data_situacao_cadastral >= "20190201" and data_situacao_cadastral <= "20200331" ')  #ganhar  cnpj_final['data_situacao_cadastral'] + cnpj_final['codigo_ibge_ajustado'] 
cnpj_final_abertas_com_covid = cnpj_final_abertas_total.query('data_situacao_cadastral >= "20200401" and data_situacao_cadastral <= "20210531" ') #ganhar "20210531" + cnpj_final['codigo_ibge_ajustado']
cnpj_final_abertas_sem_covid = cnpj_final_abertas_total.query('data_situacao_cadastral >= "20190201" and data_situacao_cadastral <= "20200331" ')  #ganhar  cnpj_final['data_situacao_cadastral'] + cnpj_final['codigo_ibge_ajustado'] 

In [35]:
cnpj_final_abertas_com_covid_data = "20210531" + cnpj_final_abertas_com_covid['codigo_ibge_ajustado']

In [36]:
cnpj_final_abertas_com_covid = pd.concat([cnpj_final_abertas_com_covid, cnpj_final_abertas_com_covid_data], axis=1) # Criação da coluna "cnpj_final_abertas_com_covid" para permitir a inclusão das informações de COVID para as empresas abertas durante o período de COVID

In [37]:
cnpj_final_abertas_com_covid.columns = ['cnpj_basico','cnpj_ordem','cnpj_dv','matriz_filial','codigo_situacao_cadastral','data_situacao_cadastral','data_inicio_atividade','cnae_principal','codigo_municipio_siafi','capital_social','descricao_municipio','codigo_ibge','codigo_ibge_ajustado','regiao','estado','populacaoTCU2019', 'interior_metropolitana',	'codigo_ibge_data']

In [38]:
cnpj_final_demais = pd.concat([cnpj_final_fechadas_com_covid, cnpj_final_fechadas_sem_covid, cnpj_final_abertas_sem_covid], axis=0)

In [39]:
cnpj_final_demais_data = cnpj_final_demais['data_situacao_cadastral'] + cnpj_final_demais['codigo_ibge_ajustado']

In [40]:
cnpj_final_demais = pd.concat([cnpj_final_demais, cnpj_final_demais_data], axis=1) # Criação da coluna "cnpj_final_abertas_com_covid" para permitir a inclusão das informações de COVID das demais empresas

In [41]:
cnpj_final_demais.columns = ['cnpj_basico','cnpj_ordem','cnpj_dv','matriz_filial','codigo_situacao_cadastral','data_situacao_cadastral','data_inicio_atividade','cnae_principal','codigo_municipio_siafi','capital_social','descricao_municipio','codigo_ibge','codigo_ibge_ajustado','regiao','estado','populacaoTCU2019', 'interior_metropolitana',	'codigo_ibge_data']

In [42]:
cnpj_final = pd.concat([cnpj_final_demais, cnpj_final_abertas_com_covid], axis=0)

In [43]:
cnpj_final.reset_index(inplace=True, drop=True)

In [44]:
codigo_ibge_data_covid = pd.DataFrame(cnpj_final['codigo_ibge_data'])

In [45]:
codigo_ibge_data_covid  = pd.merge(codigo_ibge_data_covid, covid_brasil_todas, how='left', on='codigo_ibge_data')

In [46]:
codigo_ibge_data_covid = codigo_ibge_data_covid.drop(['codigo_ibge_data','regiao','estado','codigo_ibge_ajustado','data_covid','populacaoTCU2019','casosNovos','obitosNovos','interior_metropolitana'], axis=1) #eliminação das colunas desnecessárias

In [47]:
codigo_ibge_data_covid.columns = ['casosAcumulado_covid','obitosAcumulado_covid','casos_acumulados_porcentagem_covid','obitos_acumulados_porcentagem_covid']

In [48]:
cnpj_final = pd.concat([cnpj_final, codigo_ibge_data_covid], axis=1) # concatenação dos dataset´s pelo codigo_ibge_data_covid

# **DATASET IDH**

Neste dataset iremos buscar as informações do IDH DO MUNICIPIO, IDHM EDUCACAO DO MUNICIPIO, IDHM LONGEVIDADE DO MUNICIPIO e IDHM RENDA DO MUNICIPIO.


*   IDH DO MUNICIPIO : O Índice de Desenvolvimento Humano (IDH) é uma unidade de medida utilizada para aferir o grau de desenvolvimento de uma determinada sociedade nos quesitos de educação, saúde e renda.
O IDH é uma referência numérica que varia entre 0 e 1. Quanto mais próximo de zero, menor é o indicador para os quesitos de saúde, educação e renda. Quanto mais próximo de 1, melhores são as condições para esses quesitos.


*   IDHM EDUCACAO DO MUNICIPIO : O indicador educação refere-se à quantidade média de anos de estudo de uma população. Entende-se que, quanto maior for o tempo de permanência de uma população na escola, melhores serão as chances de desenvolvimento para esse país.
*   IDHM LONGEVIDADE DO MUNICIPIO : Na variável saúde, avalia-se basicamente a taxa de expectativa de vida dos cidadãos de cada país participante. Entende-se que, quanto maior for essa taxa, melhores serão as condições de vida de seus habitantes.
*   IDHM RENDA DO MUNICIPIO : No quesito renda, mede-se o valor médio do rendimento dos cidadãos com base na média do Produto Interno Bruto (PIB), que é a soma de toda a riqueza produzida por um país em determinado período (normalmente anual) dividida pelo número de habitantes.

-------


Atlas Brasil - http://www.atlasbrasil.org.br/ranking

In [49]:
idh_municipios = pd.read_csv('//content/drive/MyDrive//TCC - PUC/IDH por Município.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0,decimal=',')

In [50]:
idh_municipios.shape

(5565, 10)

In [51]:
idh_municipios.head()

,Territorialidade,CODIGO IBGE,Posicao IDHM,IDHM,Posicao IDHM Educacao,IDHM Educacao,Posicao IDHM Longevidade,IDHM Longevidade,Posicao IDHM Renda,IDHM Renda
0,ABADIA DE GOIAS,5200050,117,"0,708",155,"0,622",151,"0,687",65,"0,83"
1,ABADIA DOS DOURADOS,3100104,136,"0,689",214,"0,563",145,"0,693",56,"0,839"
2,ABADIANIA,5200100,136,"0,689",198,"0,579",167,"0,671",54,"0,841"
3,ABAETE,3100203,127,"0,698",221,"0,556",118,"0,72",47,"0,848"
4,ABAETETUBA,1500107,197,"0,628",240,"0,537",259,"0,579",97,"0,798"


In [52]:
idh_municipios = idh_municipios.drop(['Territorialidade','Posicao IDHM','Posicao IDHM Educacao','Posicao IDHM Longevidade','Posicao IDHM Renda'], axis=1) #eliminação das colunas desnecessárias/redundantes

In [53]:
idh_municipios.columns = ['codigo_ibge', 'idh_municipio', 'idh_educacao', 'idh_longevidade','idh_renda']

In [54]:
cnpj_final = pd.merge(cnpj_final, idh_municipios, how='left', on='codigo_ibge') # concatenação dos dataset´s pelo codigo_ibge

# **DATASET PIB**

PIB é a sigla para Produto Interno Bruto, que, em linhas gerais, é um indicador econômico bastante utilizado na Macroeconomia (ramo das Ciências Econômicas) que apresenta a soma de todos os bens e serviços produzidos em uma área geográfica em um determinado período (podendo ser um ano ou um trimestre). Sendo assim, o PIB representa a dinâmica econômica do lugar, apontando o possível crescimento da economia.

*   Neste dataset iremos incluir os dados do PIB e PIB PER CAPITA do município de cada CNPJ´s estudado.

------------

Produto Interno Bruto dos Municípios | IBGE - https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9088-produto-interno-bruto-dos-municipios.html?t=downloads&c=1100023



In [55]:
pib_municipios = pd.read_csv('//content/drive/MyDrive/TCC - PUC/PIB dos Municípios - base de dados 2010-2018.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0,decimal=',')

In [56]:
pib_municipios.shape

(5570, 6)

In [57]:
pib_municipios.head()

,Sigla da Unidade da Federação,Código do Município,Nome do Município,PIB,PIB PER CAPITA,Ano
0,AC,1200013,Acrelândia,"236.379.386,00","15.737,64",2018
1,AC,1200054,Assis Brasil,"91.485.893,00","12.532,31",2018
2,AC,1200104,Brasiléia,"417.839.292,00","16.165,25",2018
3,AC,1200138,Bujari,"157.797.041,00","15.606,47",2018
4,AC,1200179,Capixaba,"170.402.523,00","14.874,52",2018


In [58]:
pib_municipios = pib_municipios.drop(['Sigla da Unidade da Federação','Nome do Município','Ano'], axis=1) #eliminação das colunas desnecessárias/redundantes

In [59]:
pib_municipios.columns = ['codigo_ibge', 'pib', 'pib_per_capita']

In [60]:
cnpj_final = pd.merge(cnpj_final, pib_municipios, how='left', on='codigo_ibge') # concatenação dos dataset´s pelo codigo_ibge

# **DATASET CNAE**

A CNAE é a classificação de atividades econômicas oficialmente adotada pelo Sistema Estatístico Nacional
e pelos órgãos gestores de cadastros e registros da Administração Pública do país.
A CNAE é uma classificação hierarquizada em cinco níveis – seções, divisões, grupos, classes e subclasses.
O quinto nível, o de subclasses, corresponde ao detalhamento usado para a identificação econômica das
unidades de produção, normalmente constituídas como pessoa jurídica ou profissionais autônomos, em
cadastros e registros da Administração Pública, nas três esferas de governo.

*   cnae_principal : representa a subclasse da classificação. É o nível mais detalhado da classificação.
*   cnae_divisao : representa a divisão na classificação CNAE.
*   cnae_secao_numero : representação numérica da classificação da seção do CNAE que é em letras.
*   cnae_secao : representa a seção na classificação CNAE a qual pertence o CNPJ. É composto por 21 seções, nível mais elevado da classificação CNAE, o qual será considerado no nosso estudo para observar o impacto do COVID em setores da economia.
*   cnae_descricao : descrição da seção em análise.

-----------

https://cnae.ibge.gov.br/?option=com_cnae&view=estrutura&Itemid=6160&chave=&tipo=cnae&versao_classe=7.0.0&versao_subclasse=9.1.0

In [61]:
cnae = pd.read_csv('//content/drive/MyDrive/TCC - PUC/CLASSIFICACAO CNAE POR SECAO.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)

In [62]:
cnae.columns = ['cnae_divisao', 'cnae_secao','cnae_descricao']

In [63]:
cnae.shape

(87, 3)

In [64]:
cnae.head()

,cnae_divisao,cnae_secao,cnae_descricao
0,1,a,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
1,2,a,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
2,3,a,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
3,5,b,INDÚSTRIAS EXTRATIVAS
4,6,b,INDÚSTRIAS EXTRATIVAS


In [65]:
cnae_divisao = cnpj_final['cnae_principal'].map(lambda x: str(x)[:-5])

In [66]:
cnae_divisao = cnae_divisao.astype(int)

In [67]:
cnae['cnae_divisao'] = cnae['cnae_divisao'].astype(int)

In [68]:
cnpj_final = pd.concat([cnpj_final, cnae_divisao], axis=1) # concatenação dos dataset´s pelo cnae_divisao

In [69]:
cnpj_final.columns = ['cnpj_basico','cnpj_ordem','cnpj_dv','matriz_filial','codigo_situacao_cadastral','data_situacao_cadastral','data_inicio_atividade','cnae_principal','codigo_municipio_siafi','capital_social','descricao_municipio','codigo_ibge','codigo_ibge_ajustado','regiao','estado','populacaoTCU2019','interior_metropolitana','codigo_ibge_data','casosAcumulado_covid','obitosAcumulado_covid','casos_acumulados_porcentagem_covid','obitos_acumulados_porcentagem_covid','idh_municipio','idh_educacao','idh_longevidade','idh_renda','pib','pib_per_capita','cnae_divisao']

In [70]:
cnpj_final = pd.merge(cnpj_final, cnae, how='left', on='cnae_divisao') # merge dos datasets com as informações do CNAE

# **PREPARAÇÃO FINAL DO DATASET**

## **Criação da coluna alvo do modelo de Machine Learning**


*   A coluna "situacao_cadastral_target" contendo a seguinte informação:
*   CNPJ´s FECHADOS (situação cadastral igual a "8"): valor um (1)
*   CNPJ´s ABERTOS (situação cadastral igual a "2"): valor zero (0)

In [71]:
data = {
'codigo_situacao_cadastral': ['02', '08'],
'situacao_cadastral_target': ['0', '1']
} #Criação da coluna alvo do modelo de Machine Learning

In [72]:
situacao_cadastral = pd.DataFrame(data, columns=['codigo_situacao_cadastral','situacao_cadastral_target'])

In [73]:
cnpj_final = pd.merge(cnpj_final, situacao_cadastral, how='left', on='codigo_situacao_cadastral')

## **Criação da coluna contendo somente o ANO e MÊS da Situação Cadastral**

In [74]:
cnpj_final['data_situacao_cadastral_ano_mes'] = pd.DataFrame (cnpj_final['data_situacao_cadastral'].map(lambda x: str(x)[:-2])) # Criação da coluna contendo o ano e mês da ocorrência da mudança da situação cadastral

In [75]:
cnpj_final = cnpj_final.drop(['codigo_ibge_ajustado','codigo_ibge_data'], axis=1) #eliminação das colunas desnecessárias

## **Ajustes na formatação das variáveis númericas com pontos decimais e vírgulas**

In [76]:
cnpj_final['pib'] = cnpj_final['pib'].str.replace('.', '')
cnpj_final['pib_per_capita'] = cnpj_final['pib_per_capita'].str.replace('.', '')
cnpj_final['capital_social'] = cnpj_final['capital_social'].str.replace(',', '.')
cnpj_final['idh_municipio'] = cnpj_final['idh_municipio'].str.replace(',', '.')
cnpj_final['idh_educacao'] = cnpj_final['idh_educacao'].str.replace(',', '.')
cnpj_final['idh_longevidade'] = cnpj_final['idh_longevidade'].str.replace(',', '.')
cnpj_final['idh_renda'] = cnpj_final['idh_renda'].str.replace(',', '.')
cnpj_final['pib'] = cnpj_final['pib'].str.replace(',', '.')
cnpj_final['pib_per_capita'] = cnpj_final['pib_per_capita'].str.replace(',', '.')

## **Gravação do Dataset**

In [77]:
cnpj_final = cnpj_final[[
    'cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'matriz_filial',  'capital_social',
    'codigo_situacao_cadastral',  
    'data_inicio_atividade', 'data_situacao_cadastral',  'data_situacao_cadastral_ano_mes', 'situacao_cadastral_target',
    'codigo_ibge', 'codigo_municipio_siafi', 'descricao_municipio',  'regiao',  'estado', 'interior_metropolitana',
    'casosAcumulado_covid',  'obitosAcumulado_covid',  'casos_acumulados_porcentagem_covid',  'obitos_acumulados_porcentagem_covid',
    'idh_municipio', 'idh_educacao', 'idh_longevidade','idh_renda',
    'populacaoTCU2019', 'pib', 'pib_per_capita',
    'cnae_principal',  'cnae_divisao',  'cnae_secao',  'cnae_descricao'
]]

In [78]:
cnpj_final.shape

(11157257, 31)

In [79]:
cnpj_final.head()

,cnpj_basico,cnpj_ordem,cnpj_dv,matriz_filial,capital_social,codigo_situacao_cadastral,data_inicio_atividade,data_situacao_cadastral,data_situacao_cadastral_ano_mes,situacao_cadastral_target,codigo_ibge,codigo_municipio_siafi,descricao_municipio,regiao,estado,interior_metropolitana,casosAcumulado_covid,obitosAcumulado_covid,casos_acumulados_porcentagem_covid,obitos_acumulados_porcentagem_covid,idh_municipio,idh_educacao,idh_longevidade,idh_renda,populacaoTCU2019,pib,pib_per_capita,cnae_principal,cnae_divisao,cnae_secao,cnae_descricao
0,29880797,0001,40,1,3500.00,08,20180308,20200507,202005,1,3170206,5403,UBERLANDIA,Sudeste,MG,0,199.0,9.0,0.029,0.001,0.789,0.716,0.776,0.885,691305.0,37442790976.00,54801.25,9511800,95,s,OUTRAS ATIVIDADES DE SERVIÇOS
1,28424384,0001,99,1,3000.00,08,20170815,20200610,202006,1,3304557,6001,RIO DE JANEIRO,Sudeste,RJ,1,38960.0,4716.0,0.580,0.070,0.799,0.719,0.84,0.845,6718903.0,364052058232.00,54426.08,8599604,85,p,EDUCAÇÃO
2,29881212,0001,07,1,10000.00,08,20180308,20201214,202012,1,4101200,7421,ANTONINA,Sul,PR,0,1038.0,26.0,5.469,0.137,0.687,0.564,0.686,0.838,18980.0,538253205.00,28312.72,4744099,47,g,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...
3,29881383,0001,36,1,5000.00,08,20180308,20210514,202105,1,3510500,6311,CARAGUATATUBA,Sudeste,SP,0,13239.0,354.0,10.893,0.291,0.759,0.705,0.735,0.845,121532.0,3325920049.00,27802.88,1091102,10,c,INDÚSTRIAS DE TRANSFORMAÇÃO
4,29881861,0001,08,1,100000.00,08,20180308,20200921,202009,1,4314902,8801,PORTO ALEGRE,Sul,RS,1,24446.0,926.0,1.648,0.062,0.805,0.702,0.867,0.857,1483771.0,77134613181.00,52149.66,8630504,86,q,SAÚDE HUMANA E SERVIÇOS SOCIAIS


In [80]:
cnpj_final.to_csv('/content/drive/MyDrive/TCC - PUC/dataset_final.csv', index=False, sep=';', encoding='ISO-8859-1')
